In [44]:
import sys
sys.executable

'c:\\Users\\dangn\\anaconda3\\envs\\tensorflowgpu\\python.exe'

In [45]:
import tensorflow as tf   
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random 

In [47]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [48]:
import sys
print(sys.executable)

c:\Users\dangn\anaconda3\envs\tensorflowgpu\python.exe


In [49]:
import tensorflow as tf   
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [50]:
df = pd.read_csv("datasets\covid19vaccinesbycountybydemographic.csv")

In [51]:
df = df[df['demographic_category'].str.contains("Race/Ethnicity")]

df['demographic_category'].unique()

array(['Race/Ethnicity'], dtype=object)

In [52]:
df['demographic_value'].unique()

array(['American Indian or Alaska Native', 'Asian',
       'Black or African American', 'Latino', 'Multiracial',
       'Native Hawaiian or Other Pacific Islander', 'Other Race',
       'Unknown', 'White'], dtype=object)

In [53]:
df = df[~df['demographic_value'].str.contains("Other Race|Unknown")]

df['demographic_value'].unique()

array(['American Indian or Alaska Native', 'Asian',
       'Black or African American', 'Latino', 'Multiracial',
       'Native Hawaiian or Other Pacific Islander', 'White'], dtype=object)

In [54]:
encoder = LabelEncoder()
df['county_encoded'] = encoder.fit_transform(df['county'])
df['demographic_encoded'] = encoder.fit_transform(df['demographic_value'])

In [55]:
df['total_population'] = df['est_population'] + df['est_age_12plus_pop'] + df['est_age_5plus_pop']

df['percent_fully'] = df['cumulative_fully_vaccinated'] / df['total_population']

In [56]:
df['administered_date'] = pd.to_datetime(df['administered_date'], format = '%Y-%m-%d')

df['time_since_start'] = (df['administered_date'] - df['administered_date'].min()).dt.days

In [57]:

prep_df = df[['percent_fully', 'county_encoded', 'demographic_encoded', 'time_since_start']]

prep_df

,percent_fully,county_encoded,demographic_encoded,time_since_start
24240,0.360569,0,0,1488
24241,0.360569,0,0,1487
24242,0.360569,0,0,1486
24243,0.360569,0,0,1485
24244,0.360569,0,0,1484
...,...,...,...,...
2179876,0.000000,58,6,4
2179877,0.000000,58,6,3
2179878,0.000000,58,6,2
2179879,0.000000,58,6,1


In [58]:
prep_df = prep_df[prep_df['percent_fully'] != 0]
prep_df = prep_df[prep_df['percent_fully'].notnull()]


prep_df

,percent_fully,county_encoded,demographic_encoded,time_since_start
24240,0.360569,0,0,1488
24241,0.360569,0,0,1487
24242,0.360569,0,0,1486
24243,0.360569,0,0,1485
24244,0.360569,0,0,1484
...,...,...,...,...
2179837,0.000009,58,6,43
2179838,0.000009,58,6,42
2179839,0.000009,58,6,41
2179840,0.000009,58,6,40


In [59]:
time_step = 10


grouped_county = prep_df.groupby(['county_encoded', 'demographic_encoded'])


In [60]:
X = []
y = []

for name, group in grouped_county:
    values = group[['percent_fully', 'county_encoded', 'demographic_encoded', 'time_since_start']].values

    for i in range(len(values) - time_step):
        X.append(values[i:i+time_step,:])
        y.append(values[i+time_step, 0])

X = np.array(X)
y = np.array(y)

In [61]:
X.shape

(518314, 10, 4)

In [62]:
np.set_printoptions(suppress=True)

X

array([[[   0.360569  ,    0.        ,    0.        , 1488.        ],
        [   0.360569  ,    0.        ,    0.        , 1487.        ],
        [   0.360569  ,    0.        ,    0.        , 1486.        ],
        ...,
        [   0.360569  ,    0.        ,    0.        , 1481.        ],
        [   0.360569  ,    0.        ,    0.        , 1480.        ],
        [   0.360569  ,    0.        ,    0.        , 1479.        ]],

       [[   0.360569  ,    0.        ,    0.        , 1487.        ],
        [   0.360569  ,    0.        ,    0.        , 1486.        ],
        [   0.360569  ,    0.        ,    0.        , 1485.        ],
        ...,
        [   0.360569  ,    0.        ,    0.        , 1480.        ],
        [   0.360569  ,    0.        ,    0.        , 1479.        ],
        [   0.360569  ,    0.        ,    0.        , 1478.        ]],

       [[   0.360569  ,    0.        ,    0.        , 1486.        ],
        [   0.360569  ,    0.        ,    0.        , 1485. 

In [63]:
model = Sequential()

model.add(LSTM(20, activation = "relu", input_shape = (X.shape[1], X.shape[2])))
model.add(Dense(10))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20)                2000      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,221
Trainable params: 2,221
Non-trainable params: 0
_________________________________________________________________


In [64]:
history = model.fit(X, y, epochs = 100, batch_size=32, validation_split=0.2)

Epoch 1/100
12958/12958 [==============================] - 242s 19ms/step - loss: 69.8424 - val_loss: 0.0197
Epoch 2/100
12958/12958 [==============================] - 244s 19ms/step - loss: 0.0073 - val_loss: 8.1988e-04
Epoch 3/100
12958/12958 [==============================] - 235s 18ms/step - loss: 0.0016 - val_loss: 6.5378e-04
Epoch 4/100
12958/12958 [==============================] - 239s 18ms/step - loss: 4.5172e-04 - val_loss: 6.6198e-05
Epoch 5/100
12958/12958 [==============================] - 241s 19ms/step - loss: 1.0467e-04 - val_loss: 1.5069e-05
Epoch 6/100
12958/12958 [==============================] - 240s 18ms/step - loss: 5.5526e-05 - val_loss: 1.2366e-06
Epoch 7/100
12958/12958 [==============================] - 241s 19ms/step - loss: 4.5678e-05 - val_loss: 4.3898e-05
Epoch 8/100
12958/12958 [==============================] - 238s 18ms/step - loss: 3.9580e-05 - val_loss: 1.9524e-04
Epoch 9/100
12958/12958 [==============================] - 240s 19ms/step - loss: 3.629

In [82]:
# model.save('models/ethnicity_county', save_format='tf')


INFO:tensorflow:Assets written to: models/ethnicity_county\assets
